In [27]:
from forecast.core import DataGenerator, make_photoz_bin_edges, compare_fisher_matrices, cl_get_idx_from_ij, ccl
from lumin_functions.core import def_agn_xlf, def_clusters_xlf
import matplotlib
from general_definitions import np, plt, set_mpl
set_mpl()
%matplotlib inline


set palette to mallsoft, shuffle=False
set matplotlib rc


In [28]:
fiducial_params = {'Omega_c': 0.25, 'Omega_b': 0.05,
                   'h': 0.7, 'sigma8': 0.8, 'n_s': 0.96,
                   'transfer_function': 'boltzmann_camb',
                   'baryons_power_spectrum': 'nobaryons',
                   'matter_power_spectrum': 'linear'}

fsky = 0.658/2
powspec_pars_dict_agn = {
    'slim': 1e-14,
    'l_min': 10,
    'l_max': 500,
    'log_bins': 41,
    'fsky': fsky,
    'has_rsd': True,
    'use_weighed_bias': False,
    'density_multiplier': 1.3,
    'camb_llimber': 110,
    'xlf': def_agn_xlf,
    'use_camb': True,
    'delta_i': 3,
    'remove_ignored_cells': True,
}



zmin_agn = 0.5
zmax_agn = 2.5
sigma0_agn = 0.2
f_fail_agn = 0.02
bins_agn = make_photoz_bin_edges(zmin_agn, zmax_agn, k=1, sigma_0=sigma0_agn)

powspec_pars_dict_agn['sigma_0'] = sigma0_agn
powspec_pars_dict_agn['f_fail'] = f_fail_agn

In [29]:
bins_agn

array([0.5   , 0.8   , 1.16  , 1.592 , 2.1104, 2.7325])

In [30]:
datagen_AGN = DataGenerator(
                                 fiducial_params=fiducial_params, set_name = 'AGN')

datagen_AGN.invoke(**powspec_pars_dict_agn, bin_left_edges=bins_agn, plot_dndz=False, plot_cell=False);
datagen_AGN.invert_cov();

dNdz calculating: 100%|██████████| 751/751 [00:03<00:00, 247.71it/s]


Total AGN sources: 918721 at fsky=0.329[13572 deg^2]
Photo-z parameters: self.sigma_0=0.2, self.f_fail=0.02
calculating CAMB...


calc Covariance: 100%|██████████| 15/15 [00:00<00:00, 1441.21it/s]

CAMB calculation time: 2.5309183597564697


In [31]:
F = datagen_AGN.get_Fisher_matrix(jac_step = 0.001, jac_order = 2)[0]

Start Jacobian calculation (cosmological part): jac_step=0.001, jac_order=2
Finished Jacobian calculation (cosmological part) in 25.22 seconds


In [32]:
F.check_derivatives()

True

In [34]:
F.F/F_test

array([[0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5]])

In [25]:

F_test = F.F
cov_test = np.linalg.inv(F_test)
np.log10(np.pi/np.sqrt(np.linalg.det(cov_test)))
#7.823

7.823361856167921

In [26]:
F_test

array([[ 31121.89374663, -34958.78529828,  11845.17867605,
         14029.38118693, -21396.34451013],
       [-34958.78529524,  42607.01156372, -13593.58188946,
        -15924.87519302,  26573.62669695],
       [ 11845.1786772 , -13593.58189135,   4581.67051202,
          5450.86767238,  -8936.97780045],
       [ 14029.38118651, -15924.8751941 ,   5450.86767174,
          6564.95974183, -11396.37849074],
       [-21396.34450147,  26573.6266956 ,  -8936.97779667,
        -11396.37848637,  31618.07141437]])

In [66]:
# F_calc = np.load('./data/AGN_0.2_0.01.npz')['F']
# F.F/F_calc